In [1]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import date, timedelta
import calendar

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2022, 1, 17), datetime.date(2022, 1, 16))

In [2]:
#yesterday = today - timedelta(days=4)
yesterday

datetime.date(2022, 1, 11)

In [2]:
format_dict = {'Q4':'{:.4f}','Q3':'{:.4f}','Q2':'{:.4f}','Q1':'{:.4f}','DIVIDEND':'{:.4f}',
               'qtrly':'{:.4f}','Q3':'{:.4f}','Q2':'{:.4f}','Q1':'{:.4f}','dividend':'{:.4f}',
               'amount':'{:,.2f}','net':'{:,.2f}',
              'price':'{:.2f}','unit_cost':'{:.2f}','PERCENT':'{:.2f}%','yield':'{:.2f}%','shares':'{:,}'
              }

### New dividend stock

In [ ]:
name = 'IP'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
df = pd.read_sql(sql, const)
df

In [ ]:
sql = """
INSERT INTO dividend
VALUES('IP',0.185,0,0.0,0,0.185,0,0,4500,'2022-04-08','2022-04-27','',0)
"""
rp = const.execute(sql)
rp.rowcount

### Start of Update dividend

In [ ]:
def update_dividend(q4,XDATE,PAIDDATE,actual,name):
    sql = "UPDATE dividend SET q4 = %s, dividend = q1+q2+q3+q4, XDATE = '%s', PAIDDATE = '%s', actual = %s WHERE name = '%s'"
    sql = sql % (q4,XDATE, PAIDDATE, actual,name)
    rp = const.execute(sql)
    return "Records updated = " + str(rp.rowcount)

In [ ]:
name = 'SIS'
sql = """
SELECT * FROM DIVIDEND WHERE name = '%s'"""
sql = sql % name
df = pd.read_sql(sql, const)
df['SHARES'] = df['SHARES'].astype(int)
df.style.format(format_dict)

In [ ]:
q4 = 0.10
XDATE = '2021-05-21'
PAIDDATE = '2021-05-21'
actual = 0

update_dividend(q4,XDATE,PAIDDATE,actual,name)

In [3]:
def update_dividend(shares,q4,actual,name):
    sql = "UPDATE dividend SET shares = %s, q4 = %s, dividend=q1+q2+q3+q4, actual = %s WHERE name = '%s'"
    sql = sql % (shares, q4, actual, name)
    rp = const.execute(sql)
    return "Records updated = " + str(rp.rowcount)

In [7]:
name = 'KBANK'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
df = pd.read_sql(sql, const)
df['SHARES'] = df['SHARES'].astype(int)
df.style.format(format_dict)

,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,KBANK,2.5000,0.0000,0.5000,0.0000,3.0000,0.000000,0.00%,1600,2022-05-07,2022-05-07,,0


In [5]:
shares = 1600
q4 = 2.5
actual = 0

In [6]:
update_dividend(shares,q4,actual,name)

'Records updated = 1'

### Delete dividend record

In [ ]:
sql = '''
DELETE FROM dividend
WHERE name IN ("IP")
'''
rp = const.execute(sql)
rp.rowcount

### Start of output to cloud

In [20]:
sql = """
SELECT Y.NAME AS name,  Q4 AS qtrly,
SHARES AS shares, Q4*SHARES AS amount, Q4*SHARES*.9 AS net,
xdate as xd_date, paiddate AS pay_date, P.price AS price, 
dividend, ROUND(dividend/P.PRICE*100,2) AS yield, actual
FROM dividend AS Y, price AS P
WHERE Y.name = P.name 
AND shares > 0 AND Q4 > 0 
AND P.date = '%s'
ORDER BY name
"""
sql = sql % today 
print(sql)


SELECT Y.NAME AS name,  Q4 AS qtrly,
SHARES AS shares, Q4*SHARES AS amount, Q4*SHARES*.9 AS net,
xdate as xd_date, paiddate AS pay_date, P.price AS price, 
dividend, ROUND(dividend/P.PRICE*100,2) AS yield, actual
FROM dividend AS Y, price AS P
WHERE Y.name = P.name 
AND shares > 0 AND Q4 > 0 
AND P.date = '2022-01-12'
ORDER BY name



In [24]:
dividend = pd.read_sql(sql, const)
dividend['shares'] = dividend['shares'].astype('int64')
#dividend.set_index('name', inplace=True)
dividend.sort_values(['yield'],ascending=False).style.format(format_dict)

,name,qtrly,shares,amount,net,xd_date,pay_date,price,dividend,yield,actual
22,STA,1.7500,"20,000","35,000.00","31,500.00",2021-12-08,2021-12-08,31.50,5.2500,16.67%,0
13,NOBLE,0.5000,"63,000","31,500.00","28,350.00",2022-05-10,2021-09-07,6.40,0.8500,13.28%,0
25,TMT,0.5000,"27,000","13,500.00","12,150.00",2021-09-10,2021-09-10,10.30,1.1000,10.68%,0
11,MCS,0.6000,"60,000","36,000.00","32,400.00",2022-03-15,2021-12-09,14.90,1.3700,9.19%,0
8,JASIF,0.2500,"100,000","25,000.00","22,500.00",2022-02-24,2021-12-08,10.50,0.9500,9.05%,0
5,HREIT,0.1718,"30,000","5,154.00","4,638.60",2022-03-01,2021-12-23,9.00,0.6932,7.70%,0
3,DIF,0.2610,"60,000","15,660.00","14,094.00",2022-02-23,2021-12-03,14.20,1.0440,7.35%,0
16,RATCH,1.2500,"9,000","11,250.00","10,125.00",2022-03-02,2021-09-23,45.00,3.2500,7.22%,0
28,WHART,0.1915,"10,000","1,915.00","1,723.50",2022-03-04,2021-12-08,12.10,0.8298,6.86%,0
17,RCL,0.5000,"4,500","2,250.00","2,025.00",2021-12-03,2021-12-03,53.00,3.5000,6.60%,0


In [18]:
dividend.net.sum()

227026.35

In [19]:
file_name = 'dividend-q4.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

dividend.sort_values(['name'],ascending=[True]).to_csv(output_file, index=False)
dividend.sort_values(['name'],ascending=[True]).to_csv(data_file, index=False)
dividend.sort_values(['name'],ascending=[True]).to_csv(box_file, index=False)

### End of output to cloud